In [1]:
def calc_upper_bound(path):
    history = [(0, float('inf'), -float('inf'))]
    resources = [(0, 0)]

    f = open(path)
    lines = f.readlines()
    first_line = lines[0].split()
    # Number of jobs
    nb_jobs = int(first_line[0])
    # Number of machines
    nb_machines = int(first_line[1])
    # Number of workers
    nb_workers = int(first_line[2])
    # Number of operations for each job
    nb_operations = [int(lines[j + 1].split()[0]) for j in range(nb_jobs)]

    # Constant for incompatible machines
    INFINITE = float('inf')
            
    # Processing time for each task, for each machine-worker combination

    task_processing_time = [[[[INFINITE for s in range(nb_workers)] 
                                        for m in range(nb_machines)] 
                                        for o in range(nb_operations[j])] 
                                        for j in range(nb_jobs)]

    job_op_dur_helper = {}
    job_op_machsuitable = {}
    job_op_mach_workersuitable = {}
    for j in range(nb_jobs):
        line = lines[j + 1].split()
        tmp_idx = 1
        for o in range(nb_operations[j]):
            nb_machines_operation = int(line[tmp_idx])
            tmp_idx += 1
            suitable_machine_helper = []
            for i in range(nb_machines_operation):
                machine = int(line[tmp_idx])
                suitable_machine_helper += [machine]
                nb_m_workers = int(line[tmp_idx + 1])
                tmp_idx += 2
                suitable_worker_helper = []
                for s in range(nb_m_workers):
                    worker = int(line[tmp_idx])
                    time = int(line[tmp_idx + 1])
                    tmp_idx += 2
                    task_processing_time[j][o][machine-1][worker-1] = time
                    suitable_worker_helper += [worker]
                    job_op_dur_helper[j+1,o+1,machine,worker] = time
                
                job_op_mach_workersuitable[j+1,o+1,machine] = suitable_worker_helper
            job_op_machsuitable[j+1,o+1] = suitable_machine_helper


    # Trivial upper bound for the completion times of the tasks
    L=0
    for j in range(nb_jobs):
        for o in range(nb_operations[j]):
            L += max(task_processing_time[j][o][m][s] for m in range(nb_machines) for s in range(nb_workers) if task_processing_time[j][o][m][s] != INFINITE )
    return L

from benchmarkrewriter.benchmark_parser import WorkerBenchmarkParser
def calc_lower_bound(path):
    parser = WorkerBenchmarkParser()
    encoding = parser.parse_benchmark(path)
    durations = encoding.durations()
    js = encoding.job_sequence()
    fastest_times = [0] * len(set(js))
    paths = [None] * len(set(js))
    prev = 0
    index = 0
    for i in range(len(js)):
        job = js[i]
        if job != prev:
            index += 1
            prev = job
        op_durations = durations[i]
        min_duration = float('inf')
        for j in range(len(op_durations)):
            for k in range(len(op_durations[j])):
                if op_durations[j][k] > 0 and op_durations[j][k] < min_duration:
                    min_duration = op_durations[j][k]
                    if not paths[index]:
                        paths[index] = []
                    paths[index].append((j, k))
        fastest_times[index] += min_duration
    return max(fastest_times)

In [2]:
def normalize(x, lb, ub):
    return (x - lb) / (ub - lb)

In [3]:
import os
base_path = r'/home/dhu/Downloads/benchmarks_with_workers'
files = os.listdir(base_path)
results = []
for file in files:
    path = base_path + '/' + file
    ub = calc_upper_bound(path)
    lb = calc_lower_bound(path)
    name = file.split('_')
    name = ''.join(name[1:-1])
    results.append(f'{name};{lb};{ub}\n')



In [4]:
with open('bounds.txt', 'a') as f:
    f.writelines(results)